In [1]:
from google.colab import drive
drive.mount("/content/drive")
#! ls "/content/drive/My Drive"

Mounted at /content/drive


In [2]:
import os 
os.getcwd()

'/content'

In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/AI比赛/2.阿里云安全检测/') 

In [4]:
!pwd

/content/drive/My Drive/Colab Notebooks/AI比赛/2.阿里云安全检测


In [ ]:
#使用fasttext
#score=1.648646(epoch=50)   score = 1.390861（epoch=500)  score = 1.378812(epoch=800)

In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import pickle

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras import regularizers

In [7]:
with open('df_train2.pkl', 'rb') as f:
  df_train = pickle.load(f)

with open('df_test2.pkl', 'rb') as f:
  df_test = pickle.load(f)

In [8]:
df_train

,file_id,label,api_count,api_nunique,api_max,api_min,api_mean,api_std,api_ptp,tid_count,tid_nunique,tid_max,tid_min,tid_mean,tid_std,tid_ptp,index_count,index_nunique,index_max,index_min,index_mean,index_std,index_ptp,api,__exception__,__exception__ ntterminateprocess,closesocket,cocreateinstance,cocreateinstance cocreateinstance,cocreateinstance ldrloaddll,cocreateinstance ldrloaddll ldrgetprocedureaddress,cocreateinstance ntallocatevirtualmemory,cocreateinstance ntopensection,cocreateinstance ntopensection ntopensection,cocreateinstance regopenkeyexw,cocreateinstance regopenkeyexw regopenkeyexw,coinitializeex,coinitializeex coinitializeex,coinitializeex coinitializeex ldrgetdllhandle,coinitializeex ldrgetdllhandle,...,setfilepointer ntreadfile ntreadfile,setfilepointer ntreadfile setfilepointer,setfilepointer setfilepointer,setfilepointer setfilepointer ntreadfile,setfilepointer setfilepointer setfilepointer,setstdhandle,setstdhandle ldrgetdllhandle,setstdhandle ldrgetdllhandle ldrgetdllhandle,setunhandledexceptionfilter,setunhandledexceptionfilter ntallocatevirtualmemory,setunhandledexceptionfilter ntallocatevirtualmemory ntallocatevirtualmemory,setunhandledexceptionfilter ntopenthread,setunhandledexceptionfilter ntopenthread ldrgetdllhandle,setwindowshookexa,shellexecuteexw,shgetfolderpathw,shgetfolderpathw seterrormode,shgetfolderpathw seterrormode getfileattributesw,shgetfolderpathw shgetfolderpathw,shgetfolderpathw shgetfolderpathw seterrormode,sizeofresource,socket,startservicea,startservicea regopenkeyexa,startservicea regopenkeyexa regsetvalueexa,thread32first,thread32first thread32next,thread32first thread32next thread32next,thread32next,thread32next ntclose,thread32next ntclose __exception__,thread32next ntclose ldrgetdllhandle,thread32next thread32next,thread32next thread32next ntclose,thread32next thread32next thread32next,uuidcreate,uuidcreate ntopenthread,uuidcreate ntopenthread ldrunloaddll,writeconsolew,wsastartup
0,1,5,6786,116,298,6,171.965223,67.767773,292,6786,11,2812,2488,2537.987622,83.881299,324,6786,5001,5000,0,2000.806955,1510.694221,5000,LdrLoadDll LdrGetProcedureAddress LdrGetProced...,0.000314,0.000426,0.000458,0.004826,0.002013,0.000886,0.000891,0.000000,0.0000,0.000000,0.000912,0.0,0.002239,0.000392,0.000506,0.000434,...,0.006420,0.007465,0.013174,0.009619,0.004488,0.0,0.0,0.0,0.000218,0.000436,0.000482,0.000000,0.000000,0.001364,0.0,0.001117,0.000000,0.0,0.000000,0.000000,0.000357,0.001691,0.000000,0.0,0.0,0.000954,0.000954,0.000954,0.504739,0.000957,0.000482,0.000479,0.503785,0.000982,0.50283,0.000776,0.000477,0.00048,0.000000,0.004209
1,2,2,816,30,298,89,159.696078,35.746150,209,816,4,2604,2320,2471.000000,101.506783,284,816,204,203,0,101.500000,58.925137,203,GetSystemTimeAsFileTime SetUnhandledExceptionF...,0.000000,0.000000,0.057138,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.013614,0.027181,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052686,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.040358
2,3,0,463,42,258,9,164.948164,51.508095,249,463,1,2208,2208,2208.000000,0.000000,0,463,463,462,0,231.000000,133.800847,462,NtAllocateVirtualMemory SetErrorMode LdrLoadDl...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.005474,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.010007,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
3,4,0,2046,51,257,9,154.939883,48.451188,248,2046,9,2980,2284,2430.727273,150.460506,696,2046,1028,1027,0,

In [9]:
df_test

,file_id,api_count,api_nunique,api_max,api_min,api_mean,api_std,api_ptp,tid_count,tid_nunique,tid_max,tid_min,tid_mean,tid_std,tid_ptp,index_count,index_nunique,index_max,index_min,index_mean,index_std,index_ptp,api,__exception__,__exception__ ntterminateprocess,closesocket,cocreateinstance,cocreateinstance cocreateinstance,cocreateinstance ldrloaddll,cocreateinstance ldrloaddll ldrgetprocedureaddress,cocreateinstance ntallocatevirtualmemory,cocreateinstance ntopensection,cocreateinstance ntopensection ntopensection,cocreateinstance regopenkeyexw,cocreateinstance regopenkeyexw regopenkeyexw,coinitializeex,coinitializeex coinitializeex,coinitializeex coinitializeex ldrgetdllhandle,coinitializeex ldrgetdllhandle,coinitializeex ldrgetdllhandle ldrgetprocedureaddress,...,setfilepointer ntreadfile ntreadfile,setfilepointer ntreadfile setfilepointer,setfilepointer setfilepointer,setfilepointer setfilepointer ntreadfile,setfilepointer setfilepointer setfilepointer,setstdhandle,setstdhandle ldrgetdllhandle,setstdhandle ldrgetdllhandle ldrgetdllhandle,setunhandledexceptionfilter,setunhandledexceptionfilter ntallocatevirtualmemory,setunhandledexceptionfilter ntallocatevirtualmemory ntallocatevirtualmemory,setunhandledexceptionfilter ntopenthread,setunhandledexceptionfilter ntopenthread ldrgetdllhandle,setwindowshookexa,shellexecuteexw,shgetfolderpathw,shgetfolderpathw seterrormode,shgetfolderpathw seterrormode getfileattributesw,shgetfolderpathw shgetfolderpathw,shgetfolderpathw shgetfolderpathw seterrormode,sizeofresource,socket,startservicea,startservicea regopenkeyexa,startservicea regopenkeyexa regsetvalueexa,thread32first,thread32first thread32next,thread32first thread32next thread32next,thread32next,thread32next ntclose,thread32next ntclose __exception__,thread32next ntclose ldrgetdllhandle,thread32next thread32next,thread32next thread32next ntclose,thread32next thread32next thread32next,uuidcreate,uuidcreate ntopenthread,uuidcreate ntopenthread ldrunloaddll,writeconsolew,wsastartup
0,1,97,15,262,13,155.989691,50.338954,249,97,4,2568,2332,2518.639175,57.218548,236,97,31,30,0,14.443299,9.210466,30,RegOpenKeyExA CopyFileA OpenSCManagerA CreateS...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.117531,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,2,1361,40,261,6,138.025716,49.557795,255,1361,7,2748,2472,2576.864071,104.399149,276,1361,681,680,0,339.750184,196.515744,680,RegOpenKeyExA RegOpenKeyExA RegOpenKeyExA NtCl...,0.000000,0.000000,0.000000,0.001948,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.012048,0.009502,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045375,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,3,16,9,257,16,111.375000,75.152179,241,16,1,2344,2344,2344.000000,0.000000,0,16,16,15,0,7.500000,4.760952,15,GetSystemTimeAsFileTime SetUnhandledExceptionF...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.103336,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,4,193,34,262,13,172.217617,45.377632,249,193,3,2584,2452,2476.331606,50.951508,132,193,193,192,0,96.000000,55.858452,192,LdrLoadDll LdrGetProcedureAddress CreateToolhe...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0000

In [11]:
import gc
#del train, test
gc.collect()

88

In [12]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer

In [13]:
tokenizer.fit_on_texts(df_train['api'].tolist())

In [14]:
tokenizer.fit_on_texts(df_test['api'].tolist())

In [15]:
vocab = tokenizer.word_index
print(len(vocab))
vocab

303


{'a': 282,
 'accept': 214,
 'assignprocesstojobobject': 269,
 'bind': 181,
 'certcontrolstore': 206,
 'certcreatecertificatecontext': 224,
 'certopenstore': 192,
 'certopensystemstorea': 270,
 'certopensystemstorew': 289,
 'closesocket': 85,
 'cocreateinstance': 46,
 'cocreateinstanceex': 185,
 'cogetclassobject': 139,
 'coinitializeex': 67,
 'coinitializesecurity': 177,
 'connect': 115,
 'controlservice': 237,
 'copyfilea': 143,
 'copyfileexw': 260,
 'copyfilew': 227,
 'couninitialize': 70,
 'createactctxw': 128,
 'createdirectoryexw': 296,
 'createdirectoryw': 93,
 'createjobobjectw': 250,
 'createprocessinternalw': 102,
 'createremotethread': 162,
 'createremotethreadex': 288,
 'createservicea': 168,
 'createservicew': 249,
 'createthread': 69,
 'createtoolhelp32snapshot': 101,
 'cryptacquirecontexta': 187,
 'cryptacquirecontextw': 190,
 'cryptcreatehash': 125,
 'cryptdecodeobjectex': 22,
 'cryptdecrypt': 121,
 'cryptencrypt': 189,
 'cryptexportkey': 196,
 'cryptgenkey': 284,
 'cryp

In [16]:
df_train['api'][:2].tolist()

['LdrLoadDll LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress LdrGetProcedureAddress NtCreateMutant NtClose NtCreateFile NtWriteFile NtClose CreateProcessInternalW NtClose NtClose LdrUnloadDll NtAllocateVirtualMemory NtFreeVirtualMemory NtAllocateVirtualMemory GetFileType GetFileType GetFileType NtAllocateVirtualMemory NtAllocateVirtualMemory NtAllocateVirtualMemory LdrGetDllHandle LdrGetProcedureAddress NtAllocateVirtualMemory SetUnhandledExceptionFilter NtAllocateVirtualMemory NtAllocateVirtualMemory GetCursorPos WSAStartup GetSystemDirectoryA GetSystemMetrics GetSystemMetrics GetSystemMetrics GetSystemMetrics SetErrorMode SetErrorMode LoadStringA SetWindowsHookExA OleInitialize NtAllocateVirtualMemory NtAllocateVirtualMemory SetWindowsHookExA NtClose NtClose NtAllocateVirtualMemory NtAllocateVirtualMemory GetSystemMetrics GetSystemMetrics FindResourc

In [17]:
#将句子用字典id替换
x_train_word_ids = tokenizer.texts_to_sequences(df_train['api'].tolist())

In [18]:
len(x_train_word_ids)

13887

In [19]:
len(x_train_word_ids[0])

6786

In [20]:
x_test_word_ids = tokenizer.texts_to_sequences(df_test['api'].tolist())
len(x_test_word_ids)

12955

In [21]:
length=[]
for i in x_train_word_ids:
  length.append(len(i))


In [22]:
len(length)

13887

In [23]:
a,b=0,0
for i in length:
  if i<6000:
    a += 1
  else:
    b +=1

In [24]:
print(a,b)

9851 4036


In [25]:
len(x_train_word_ids[1])

816

In [26]:
max_len=6000

In [27]:
#填充到固定长度，截长补短（补0）
x_train_padded_seqs = keras.preprocessing.sequence.pad_sequences(x_train_word_ids, max_len)


In [28]:
x_train_padded_seqs[:2]

array([[  6,   6,   6, ...,   8,   6,   8],
       [  0,   0,   0, ...,   2, 209,  65]], dtype=int32)

In [29]:
x_train_padded_seqs.shape

(13887, 6000)

In [30]:
x_test_padded_seqs = keras.preprocessing.sequence.pad_sequences(x_test_word_ids, max_len)

In [31]:
x_test_padded_seqs[:2]

array([[ 0,  0,  0, ...,  2,  2, 65],
       [ 0,  0,  0, ..., 19,  2,  2]], dtype=int32)

In [32]:
#针对每个file_id的apis字符串组合，通过分词转化分字典数字后，再对每个字符串组合进行截长补短后，使每个长度一致，存储起来，可直接输入模型
with open('x_train_padded_seqs.pkl', 'wb') as f:
  pickle.dump(x_train_padded_seqs, f)

with open('x_test_padded_seqs.pkl', 'wb') as f:
  pickle.dump(x_test_padded_seqs, f)

with open('vocab.pkl', 'wb') as f:
  pickle.dump(vocab, f)


In [33]:
df_train_y = df_train['label']
df_train_y

0        5
1        2
2        0
3        0
4        0
        ..
13882    2
13883    5
13884    0
13885    1
13886    2
Name: label, Length: 13887, dtype: int64

In [34]:
label = to_categorical(df_train_y)
label

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [35]:
label.shape

(13887, 8)

In [36]:
def fasttext():
  model = Sequential([
          keras.layers.Embedding(len(vocab)+1, 256, input_length=max_len),
          keras.layers.GlobalAveragePooling1D(),
          keras.layers.Dense(8, activation='softmax')
          ])
  return model

#from skleran.model_selection import train_test_split
X, y = x_train_padded_seqs, label
#X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2,random_state=2001)
model = fasttext()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam' )
history = model.fit(X, y,batch_size=1024, epochs=1)

14/14 [==============================] - 55s 4s/step - loss: 2.0258 - accuracy: 0.3172


In [37]:
history = model.fit(X, y, batch_size=1024, epochs=300)

Epoch 1/300
14/14 [==============================] - 51s 4s/step - loss: 1.8648 - accuracy: 0.5058
Epoch 2/300
14/14 [==============================] - 51s 4s/step - loss: 1.7076 - accuracy: 0.5035
Epoch 3/300
14/14 [==============================] - 55s 4s/step - loss: 1.5724 - accuracy: 0.5018
Epoch 4/300
14/14 [==============================] - 57s 4s/step - loss: 1.4801 - accuracy: 0.5039
Epoch 5/300
14/14 [==============================] - 51s 4s/step - loss: 1.4197 - accuracy: 0.5204
Epoch 6/300
14/14 [==============================] - 51s 4s/step - loss: 1.3752 - accuracy: 0.5386
Epoch 7/300
14/14 [==============================] - 51s 4s/step - loss: 1.3412 - accuracy: 0.5481
Epoch 8/300
14/14 [==============================] - 51s 4s/step - loss: 1.3133 - accuracy: 0.5563
Epoch 9/300
14/14 [==============================] - 50s 4s/step - loss: 1.2902 - accuracy: 0.5610
Epoch 10/300
14/14 [==============================] - 52s 4s/step - loss: 1.2699 - accuracy: 0.5643
Epoch 11/

In [38]:
history = model.fit(X, y, batch_size=1024, epochs=200)

Epoch 1/200
14/14 [==============================] - 52s 4s/step - loss: 0.7812 - accuracy: 0.7135
Epoch 2/200
14/14 [==============================] - 52s 4s/step - loss: 0.7808 - accuracy: 0.7175
Epoch 3/200
14/14 [==============================] - 51s 4s/step - loss: 0.7804 - accuracy: 0.7144
Epoch 4/200
14/14 [==============================] - 51s 4s/step - loss: 0.7798 - accuracy: 0.7189
Epoch 5/200
14/14 [==============================] - 52s 4s/step - loss: 0.7807 - accuracy: 0.7113
Epoch 6/200
14/14 [==============================] - 52s 4s/step - loss: 0.7793 - accuracy: 0.7180
Epoch 7/200
14/14 [==============================] - 51s 4s/step - loss: 0.7789 - accuracy: 0.7140
Epoch 8/200
14/14 [==============================] - 51s 4s/step - loss: 0.7786 - accuracy: 0.7179
Epoch 9/200
14/14 [==============================] - 52s 4s/step - loss: 0.7781 - accuracy: 0.7144
Epoch 10/200
14/14 [==============================] - 52s 4s/step - loss: 0.7774 - accuracy: 0.7178
Epoch 11/

In [46]:
history = model.fit(X, y, batch_size=1024, epochs=300)

Epoch 1/300
14/14 [==============================] - 56s 4s/step - loss: 0.7249 - accuracy: 0.7359
Epoch 2/300
14/14 [==============================] - 57s 4s/step - loss: 0.7253 - accuracy: 0.7365
Epoch 3/300
14/14 [==============================] - 55s 4s/step - loss: 0.7246 - accuracy: 0.7372
Epoch 4/300
14/14 [==============================] - 55s 4s/step - loss: 0.7244 - accuracy: 0.7369
Epoch 5/300
14/14 [==============================] - 55s 4s/step - loss: 0.7243 - accuracy: 0.7369
Epoch 6/300
14/14 [==============================] - 56s 4s/step - loss: 0.7239 - accuracy: 0.7369
Epoch 7/300
14/14 [==============================] - 56s 4s/step - loss: 0.7238 - accuracy: 0.7369
Epoch 8/300
14/14 [==============================] - 55s 4s/step - loss: 0.7243 - accuracy: 0.7377
Epoch 9/300
14/14 [==============================] - 55s 4s/step - loss: 0.7238 - accuracy: 0.7369
Epoch 10/300
14/14 [==============================] - 53s 4s/step - loss: 0.7230 - accuracy: 0.7369
Epoch 11/

In [47]:
result = model.predict(x_test_padded_seqs) #找到probability
result

array([[2.5321848e-03, 8.7251246e-02, 4.4921819e-02, ..., 2.1228361e-01,
        9.0703271e-02, 3.8866550e-01],
       [3.1512544e-01, 4.8550215e-13, 2.0164171e-05, ..., 6.3117659e-01,
        3.7315026e-02, 1.6361790e-02],
       [9.0172559e-01, 8.4197354e-03, 1.7686144e-02, ..., 5.5410758e-02,
        2.6906948e-03, 9.8654060e-03],
       ...,
       [5.0128169e-06, 5.1581259e-07, 2.0086206e-02, ..., 6.3939941e-01,
        2.4947473e-01, 7.9372026e-02],
       [1.2686653e-03, 3.4140566e-08, 9.2785235e-04, ..., 9.9593413e-01,
        1.3830947e-03, 4.3800980e-04],
       [1.7580037e-03, 2.7007738e-05, 3.6371768e-02, ..., 9.2617899e-01,
        1.5445283e-02, 1.3044909e-02]], dtype=float32)

In [48]:
result1 = pd.DataFrame(result, columns=['prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7']) 
result1['file_id'] = df_test['file_id'].values
result1

,prob0,prob1,prob2,prob3,prob4,prob5,prob6,prob7,file_id
0,0.002532,8.725125e-02,0.044922,1.400241e-01,3.361820e-02,0.212284,0.090703,0.388665,1
1,0.315125,4.855022e-13,0.000020,7.693811e-07,2.024413e-07,0.631177,0.037315,0.016362,2
2,0.901726,8.419735e-03,0.017686,2.307976e-03,1.893737e-03,0.055411,0.002691,0.009865,3
3,0.133989,1.427139e-02,0.133247,3.642535e-02,1.639663e-02,0.462986,0.041208,0.161477,4
4,0.998105,3.847299e-06,0.000203,9.040149e-06,9.699987e-06,0.001470,0.000022,0.000177,5
...,...,...,...,...,...,...,...,...,...
12950,0.975724,1.324850e-04,0.015119,3.611354e-04,1.926057e-04,0.002424,0.001846,0.004201,12951
12951,0.545054,2.206808e-02,0.094616,1.470788e-02,7.749232e-03,0.236713,0.015489,0.063603,12952
12952,0.000005,5.158126e-07,0.020086,1.003942e-02,1.622637e-03,0.639399,0.249475,0.079372,12953
12953,0.001269,3.414057e-08,0.000928,2.931245e-05,1.890660e-05,0.995934,0.001383,0.000438,12954


In [49]:
columns = ['file_id', 'prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7']
result1.to_csv('baseline4_fasttext_800.csv', index=False, columns=columns)

In [ ]:
x_train_padded_seqs1 = keras.preprocessing.sequence.pad_sequences(x_train_word_ids1, max_len)